In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import os

In [4]:
df = pd.read_csv('Dataset_FOOD/RAW_recipes.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset_FOOD/RAW_recipes.csv'

In [5]:
# Initialising Spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [6]:
# Expand the nutrition column into multiple labelled columns
df[['calories','total fat (%)','sugar (%)','sodium (%)','protein (%)','saturated fat (%)','carbohydrates (%)']] = df.nutrition.str.split(",",expand=True)
df['calories'] = df['calories'].apply(lambda x: x.replace('[','')) 
df['carbohydrates (%)']= df['carbohydrates (%)'].apply(lambda x: x.replace(']','')) 

NameError: name 'df' is not defined

In [15]:
# Decide whether the food is healthy based on the total % and calories. If a food has more than 2500 calories or has more than 30% of the daily reccommended intake of Sugar
# it is deemed to be unhealthy. however if there are 2 or more other factors that are more than 30% (60% for protein) than the daily recommended intake then it is deemed healthy


def healthy(row):
    count = 0
    if float(row['calories']) >2500:
        return 'No'
    if float(row['total fat (%)']) > 30:
        count += 1
    if float(row['sugar (%)']) > 30:
        count += 2
    if float(row['sodium (%)']) > 30:
        count += 1
    if float(row['protein (%)']) > 70:
        count += 1
    if float(row['saturated fat (%)']) > 30:
        count += 1
    if float(row['carbohydrates (%)']) > 30:
        count += 1
    if count >= 2:
        return 'No'
    else:
        return 'Yes'
    
# Create the label column, Healthy
df['Healthy'] = df.apply(lambda x: healthy(x), axis = 1)

In [16]:
# Randomising the dataset
df = df.sample(frac=1)

In [20]:
df=df.astype(str)

In [22]:
# Extracting a training set
df_training = df.iloc[:185310,:]

In [23]:
# Extracting a testing set
df_test = df.iloc[185310:,:]

In [24]:
# Creating a preprocessed cloumn for the recipe name with lemmatization, casefolding and stopword removal
df_test['PPName'] = df_test['name'].apply(lambda lst : " ".join(token.lemma_.lower() for token in nlp(lst) if not token.is_stop))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4776\3815969779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['PPName'] = df_test['name'].apply(lambda lst : " ".join(token.lemma_.lower() for token in nlp(lst) if not token.is_stop))


In [25]:
# Exporting Training dataset to csv
df_training.to_csv("Training_Set.csv", sep='\t')

In [ ]:
# Exporting Testing dataset to csv
df_test.to_csv("Test_set.csv", sep='\t')